In [1]:
import pandas as pd

In [7]:
contracts = pd.read_pickle('contract_data/contracts_with_text.pkl')

In [9]:
contracts.columns

Index([u'OCID', u'Category', u'Contract Name', u'Contract Identifier',
       u'Language', u'Country Name', u'Resource', u'Contract Type',
       u'Signature Date', u'Document Type', u'Government Entity',
       u'Government Identifier', u'Company Name', u'Company Address',
       u'Jurisdiction of Incorporation', u'Registration Agency',
       u'Company Number', u'Corporate Grouping', u'Participation Share',
       u'Open Corporates Link', u'Incorporation Date', u'Operator',
       u'Project Title', u'Project Identifier', u'License Name',
       u'License Identifier', u'Source Url', u'Disclosure Mode',
       u'Retrieval Date', u'Pdf Url', u'Deal Number', u'Contract Note',
       u'Matrix Page', u'Annotation Category', u'Annotation Text',
       u'Contract_Text'],
      dtype='object')

In [8]:
len(contracts)

1493

In [10]:
contracts.Language.value_counts()

en    666
fr    437
es    366
pt     20
ar      3
pl      1
Name: Language, dtype: int64

In [11]:
contracts['Contract_Text'].head()

0    ﻿                 October 2.9,2012            ...
1    PRODUCTION SHARING CONTRACT<br /><br />\r\n<br...
2    ﻿   AVENANT N°1 A LA CONVENTION DE BASE POUR  ...
3    ﻿   ANNEXE A             <br />\r<br /><br />\...
4    EX-10.7 4 a2197356zex-10_7.htm EX-10.7<br /><b...
Name: Contract_Text, dtype: object